In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
#Data paths
data_dir_train = 'data/Training data'
data_dir_val = 'data/Validation data'
data_dir_test = 'data/Test data'
#Module parametres
batch_size = 32
img_height = 180
img_width = 180
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_train,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_val,
  image_size=(img_height, img_width),
  batch_size=batch_size)
test_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir_test,
  image_size=(img_height, img_width),
  batch_size=batch_size)
class_names = train_ds.class_names
print(class_names)

Found 6238 files belonging to 4 classes.
Found 400 files belonging to 4 classes.
Found 977 files belonging to 4 classes.
['Elephant', 'Panda', 'Tiger', 'Zebra']


In [2]:

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
#Standarize data
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [3]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [4]:
num_classes = len(class_names)

model = Sequential([
 # layers.BatchNormalization(input_shape=(img_height, img_width, 3)),
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])


In [5]:
model.compile(optimizer='adamax',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 180, 180, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                  

In [6]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15
195/195 [==============================] - 72s 364ms/step - loss: 0.9458 - accuracy: 0.5938 - val_loss: 1.0663 - val_accuracy: 0.5075
Epoch 2/15
195/195 [==============================] - 70s 360ms/step - loss: 0.6246 - accuracy: 0.7589 - val_loss: 0.8932 - val_accuracy: 0.6375
Epoch 3/15
195/195 [==============================] - 71s 362ms/step - loss: 0.5432 - accuracy: 0.7934 - val_loss: 1.0822 - val_accuracy: 0.5950
Epoch 4/15
195/195 [==============================] - 71s 363ms/step - loss: 0.5059 - accuracy: 0.8091 - val_loss: 1.0094 - val_accuracy: 0.6400
Epoch 5/15
195/195 [==============================] - 71s 362ms/step - loss: 0.4598 - accuracy: 0.8262 - val_loss: 0.7799 - val_accuracy: 0.7400
Epoch 6/15
195/195 [==============================] - 69s 353ms/step - loss: 0.4236 - accuracy: 0.8424 - val_loss: 0.8855 - val_accuracy: 0.7000
Epoch 7/15
195/195 [==============================] - 69s 354ms/step - loss: 0.4002 - accuracy: 0.8544 - val_loss: 0.9260 - val_ac

In [7]:
#Testing model on test data
loss, accuracy = model.evaluate(test_ds)
print('Test accuracy:', accuracy)


31/31 [==============================] - 3s 76ms/step - loss: 0.9344 - accuracy: 0.7523
Test accuracy: 0.7523029446601868


In [8]:
model.save('model4aM.h5')